# Load Libraries

In [ ]:
import numpy as np
import pandas as pd
import json
from pathlib import Path
import re

## Load tag data

In [ ]:
file = r'./domain_instances.xls'
tagDF = pd.read_excel(file)

In [ ]:
tagDF.head()

In [ ]:
len(tagDF.Host.unique())

## Load instance data

In [ ]:
# In this file we have instance_id and instance_name
file = r'./domain_volume.xlsm'
instanceDF = pd.read_excel(file)

In [ ]:
instanceDF.head()

In [ ]:
len(instanceDF.Instance_id.unique())

In [ ]:
len(instanceDF.Instance_name.unique())

## Load ami data

In [ ]:
file = r'./ec2_ami.csv'
amiDF = pd.read_csv(file, header=None)
amiDF.columns = ['i_id', 'ami']

In [ ]:
amiDF.head()

In [ ]:
len(amiDF.i_id.unique())

## Load type & state data

In [ ]:
file = r'./all_ec2_raquel.csv'
typeDF = pd.read_csv(file, header=None)
typeDF.columns = ['i_id', 'state', 'type']

In [ ]:
typeDF.head()

In [ ]:
len(typeDF.i_id.unique())

In [ ]:
len(typeDF.type.unique())

## Create DF for instance info

In [ ]:
# of the 934 instances for our analysis (in the tag data) lets create:
# id, host (instance name), ami

# left join tag data with instance data (obtain host name and id)
merged = pd.merge(tagDF, instanceDF, how='left', left_on=['Host'], right_on=['Instance_name'])
host_id = merged[['Host', 'Instance_id']]

In [ ]:
host_id.head()

In [ ]:
len(host_id)

In [ ]:
host_id[host_id.duplicated(keep=False)] #keep=False shows all occurrences of duplicates

In [ ]:
len(host_id.Host.unique())

In [ ]:
len(host_id.Instance_id.unique())

In [ ]:
# keep only unique 
host_id = host_id.drop_duplicates()

In [ ]:
len(host_id)

In [ ]:
len(host_id.Host.unique())

In [ ]:
len(host_id.Instance_id.unique())

In [ ]:
host_id[host_id.Instance_id.duplicated(keep=False)]

In [ ]:
# left join with ami 

merged2 = pd.merge(host_id, amiDF, how='left', left_on=['Instance_id'], right_on=['i_id'])
i_ami = merged2.drop(['i_id'], axis=1)

In [ ]:
i_ami.head()

In [ ]:
len(i_ami.Host.unique())

In [ ]:
len(i_ami.Instance_id.unique())

In [ ]:
i_ami[i_ami.Instance_id.duplicated(keep=False)]

In [ ]:
len(i_ami.ami.unique())

In [ ]:
len(i_ami)

#### left join with type&state data 

In [ ]:
merged3 = pd.merge(i_ami, typeDF, how='left', left_on=['Instance_id'], right_on=['i_id'])
merged3 = merged3.drop(['i_id'], axis=1)

In [ ]:
merged3.head()

In [ ]:
len(merged3)
# Same as before so we didn't lose any data

In [ ]:
len(merged3.Instance_id.unique())

In [ ]:
len(merged3.ami.unique())

In [ ]:
len(merged3.type.unique())

In [ ]:
# Merge with OS
# But we only have OS for the 12 amis in our 372 instances in logData 

## Load log data

In [ ]:
file = r'./loggedInstances.csv'
logDF = pd.read_csv(file, header=None, skiprows=1)
#log_inst = logDF['instance']

In [ ]:
logDF.head()

In [ ]:
len(logDF)

In [ ]:
#log_inst = log_inst.drop_duplicates()

In [ ]:
#len(log_inst)

In [ ]:
# log_instDF = log_inst.to_frame()
logDF.columns = ['Host']

In [ ]:
# Merge for instanceData for 372 instances
instances372 = pd.merge(logDF, merged3, on=['Host'], how='left')

In [ ]:
instances372.head()

In [ ]:
len(instances372.ami.unique())

### Add OS

In [ ]:
# Load data
osDF = pd.read_csv('./ami_os.csv')

In [ ]:
osDF

In [ ]:
allData = pd.merge(instances372, osDF, on=['ami'], how='left')

In [ ]:
allData.head()

In [ ]:
len(allData)

In [ ]:
len(allData.Host.unique())

In [ ]:
len(allData.ami.unique())

In [ ]:
(allData.type.unique())

## Save Instance info Dataframe

In [ ]:
allData.to_csv('./dataframes/instanceData.csv',sep=',',index=False, encoding="utf-8")